In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime,date
from dateutil.relativedelta import relativedelta
import time
from tqdm import tqdm
from dateutil.parser import parse
import regex as re
from src import *
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sharks = pd.read_csv("sharks_clean.csv").drop(columns="Unnamed: 0")

# Creando nuevas columnas 

### UBICACIONES

In [ ]:
sharks = pd.read_csv("sharks_clean.csv")

In [ ]:
# Me creo un nuevo df para tratar solo las ubicaciones
sharks_places = sharks.copy()

In [ ]:
sharks_places["lat"] = np.nan
sharks_places["long"] = np.nan
sharks_places.drop(columns="Unnamed: 0",inplace=True)

In [ ]:
# Leo un csv con todos los estados de USA y su latitud y longitud
states = pd.read_csv("statelatlong.csv")
states

In [ ]:
# Añado a mi dataframe de ubicaciones, la longitud y la latitud que le correpsonde al estado
for i in sharks_places.index:
    for j in states.index:
        if str(sharks_places["area"][i]).upper().strip() == states["City"][j].upper():
            sharks_places.loc[i,"lat"] = states["Latitude"][j]
            sharks_places.loc[i,"long"] = states["Longitude"][j]

In [ ]:
sharks_places[sharks_places["country"]=="USA"].sort_values("area",ascending=False)

# Exploracion de datos

### CANTIDAD DE ATAQUES TOTALES POR DECADA

In [ ]:
# Contamos la cantidad de ataques totales, tanto letales como no
#sharks_clean.fillna("Unknown",inplace=True)
attacks = sharks_places.groupby(["fatal"]).agg({"fatal":"count"})
attacks = attacks.rename(columns={"fatal": "count"})
attacks = attacks.sort_values("count", ascending=False)
attacks

In [ ]:
# Grafica de ataques fatales
attacks.plot(title = "Type of attacks by percentage", kind = "pie", autopct='%1.1f%%', subplots=True, figsize = (15,5))
plt.show()

In [ ]:
# A partir de la columna year, creamos una columna categorica que indique la decada
sharks_places["year"].fillna(0,inplace=True)
sharks_places["decade"] = sharks_places.year.apply(lambda x: (str(x - (x%10)))[0:4]+" \' s")

In [ ]:
# Cruzamos la columna decada con la columna fatal
decades = pd.crosstab(index=sharks_places.decade[sharks_places['year']>1900], columns=sharks_places["fatal"])
# Añadimos una columna con los ataques totales
decades["TOTAL"] = decades["N"] + decades["Y"]
decades["%"] = (decades["Y"] / 5043)*100
decades
#decades["TOTAL"]

In [ ]:
# Total de casos a partir de 1900
decades["N"].sum()+decades["Y"].sum()

In [ ]:
decades[["N","Y","TOTAL"]].plot(title = "Survival per decade", figsize = (15,5), kind="line")
plt.show()

In [ ]:
decades["%"].plot(title = "Survival per decade", figsize = (15,5), kind="line")
plt.show()

### QUE ESPECIE ES LA MAS PELIGROSA

In [ ]:
species = pd.crosstab(index=sharks_places.species[sharks_places.species!="unknown"], columns=[sharks_places["fatal"]]).sort_values("N",ascending=False)
species["%"] = ((species["Y"]/species["N"])*100).round(2)
species_most = species[(species["N"]+species["Y"])>20].sort_values("%",ascending=False)
species_most

In [ ]:
species_most["%"].plot(title = "Fatality per shark specie", kind = "bar", figsize = (15,4))
plt.show()

### POR AREA

In [ ]:
areas = pd.crosstab(index=sharks_places.species[sharks_places.species!="unknown"], columns=sharks_places.area[sharks_places.country=="USA"])#.sort_values("N",ascending=False)
areas

In [ ]:
areas_df = pd.DataFrame(areas.sum().sort_values(ascending=False),columns=["Fatal cases"])
areas_df[areas_df["Fatal cases"]>4]

In [ ]:
# En cada estado, cual es el tiburon mas peligroso

In [ ]:
dicc = {}
for i in areas:
    dicc[i] = areas[i].idxmax()

df_dicc = pd.DataFrame(pd.Series(dicc),columns=["Shark type"])
sharks_per_state = pd.merge(areas_df, df_dicc, left_index = True, right_index = True, how="inner")
sharks_per_state[sharks_per_state["Fatal cases"]>4]

In [ ]:
sharks_per_state[sharks_per_state["Fatal cases"]>4].plot(title = "Letality per decade", figsize = (15,5), kind="bar")
plt.show()

### QUE RANGO DE EDAD ES EL MAS VULNERABLE

In [ ]:
sharks_places["decade_age"] = sharks_places.age.apply(lambda x: (str(x - (x%10)))[0:2]+" to "+(str(x - (x%10)+10))[0:2])

In [ ]:
decades_age = pd.crosstab(index=sharks_places.decade_age, columns=sharks_places["fatal"])
decades_age = decades_age.sort_index().drop("na to na")
decades_age

In [ ]:
decades_age.plot(title = "Survival per age", figsize = (15,5))
plt.show()